In [ ]:
# !pip install boto3 botocore

In [ ]:
import os
# Fetch credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")

# Create a Bedrock client
bedrock = boto3.client(
    service_name="bedrock",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
bedrock.list_foundation_models()

In [ ]:
import os
import boto3
import json
from botocore.exceptions import ClientError

# Fetch credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")

# Create a Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Set the model ID for embedding model
model_id = "amazon.titan-embed-text-v2:0"

# Define the input text for the embedding model
input_text = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload
embedding_request = {
    "inputText": input_text
}

# Convert the payload to JSON
request = json.dumps(embedding_request)

try:
    # Invoke the model
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=request,
        contentType="application/json",
        accept="application/json"
    )

    # Decode the response body
    raw_response = response["body"].read().decode("utf-8")
    model_response = json.loads(raw_response)

    # Extract the embedding based on the actual response structure
    embedding_result = model_response
    print("Embedding Result:", embedding_result)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

In [10]:
import os
import boto3
import json
import logging
from botocore.exceptions import ClientError

# Initialize Logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Function to Generate Text
def generate_text(inference_profile_arn, body):
    """
    Generate text using the specified inference profile.
    Args:
        inference_profile_arn (str): The ARN of the inference profile to use.
        body (str): The request body to use.
    Returns:
        JSON response from the Bedrock service.
    """
    try:
        logger.info("Invoking model with the specified inference profile.")

        # Create Bedrock Runtime client
        bedrock = boto3.client(
            service_name="bedrock-runtime",
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
            region_name=os.getenv("AWS_REGION"),
        )

        # Invoke the model
        response = bedrock.invoke_model(
            body=body, modelId=inference_profile_arn, contentType="application/json", accept="application/json"
        )

        # Parse the response
        response_body = json.loads(response["body"].read())
        return response_body

    except ClientError as e:
        logger.error(f"ClientError: {e}")
        raise

    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        raise

# Fetch inference profile ARN from environment variables
inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Define the prompt
prompt = """<s>[INST] <<SYS>>
You are an AI assistant capable of explaining complex topics at various educational levels in a simple and engaging manner. Please provide clear, concise, and age-appropriate explanations for each audience: elementary school, middle school, high school, and undergraduate students. Ensure each explanation is tailored to their level of understanding while retaining the core ideas.
<</SYS>>

Explain the concept of "Attention is All You Need," a foundational idea in Generative AI, in one paragraph for each of the following levels:
1. Elementary school students
2. Middle school students
3. High school students
4. Undergraduate students
[/INST]"""

# Model parameters
max_gen_len = 1500  # Increased token length
temperature = 0.1
top_p = 0.9

# Create the request body
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": max_gen_len,
    "temperature": temperature,
    "top_p": top_p
})

# Call the function and process the response
try:
    response = generate_text(inference_profile_arn, body)

    # Print the response details
    print("Generated Text:\n", response.get("generation", "No generation found"))
    print("Prompt Token Count:", response.get("prompt_token_count", "N/A"))
    print("Generated Token Count:", response.get("generation_token_count", "N/A"))
    print("Stop Reason:", response.get("stop_reason", "Unknown"))

except Exception as e:
    print(f"An error occurred: {e}")

INFO:__main__:Invoking model with the specified inference profile.


Generated Text:
  <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to make sure you understand. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to focus on the most important things in a conversation or a piece of text, so they can understand it better. Just like how you need to focus on your friend, computers need to focus on the right words or ideas to make sense of things.

## Step 2: Explain the concept to middle school students
You know how sometimes you're trying to do a lot of things at the same time, like studying for a test, playing a game, and chatting with friends? It can get really confusing and hard to focus. "Attention is All You Need" is a way for computers to do the same thing, but instead of focusing on multiple things